In [ ]:
import os

import pandas as pd
from tqdm.auto import tqdm

In [ ]:
df = pd.read_csv("https://zenodo.org/record/4589756/files/rcmip-emissions-annual-means-v5-1-0.csv")

In [ ]:
df

In [ ]:
bbcats = [
    "Emissions|NOx|MAGICC AFOLU|Grassland Burning",
    "Emissions|NOx|MAGICC AFOLU|Peat Burning",
    "Emissions|NOx|MAGICC AFOLU|Forest Burning",
    "Emissions|NOx|MAGICC AFOLU|Agricultural Waste Burning",
]

In [ ]:
afolucats = [
    "Emissions|NOx|MAGICC AFOLU|Grassland Burning",
    "Emissions|NOx|MAGICC AFOLU|Peat Burning",
    "Emissions|NOx|MAGICC AFOLU|Forest Burning",
    "Emissions|NOx|MAGICC AFOLU|Agricultural Waste Burning",
    "Emissions|NOx|MAGICC AFOLU|Agriculture",
]

In [ ]:
# uplift by ratio of NO2 to NO
df.loc[df["Variable"].isin(bbcats), "1750":]

In [ ]:
df.loc[df["Variable"].isin(bbcats), "1750":] = df.loc[df["Variable"].isin(bbcats), "1750":] * 46.006/30.006

In [ ]:
# add together these updated contributions to MAGICC AFOLU total
for idx, data in tqdm(df.loc[df["Variable"].isin(afolucats)].groupby(["Model", "Scenario", "Region"])):
    df.loc[
        (df["Variable"]=="Emissions|NOx|MAGICC AFOLU") &
        (df["Model"]==idx[0]) &
        (df["Scenario"]==idx[1]) &
        (df["Region"]==idx[2]),
    "1750":] = data.sum(skipna=False)["1750":].values

In [ ]:
noxcats = ["Emissions|NOx|MAGICC AFOLU", "Emissions|NOx|MAGICC Fossil and Industrial"]

In [ ]:
# add together these updated contributions to NOx total
for idx, data in tqdm(df.loc[df["Variable"].isin(noxcats)].groupby(["Model", "Scenario", "Region"])):
    df.loc[
        (df["Variable"]=="Emissions|NOx") &
        (df["Model"]==idx[0]) &
        (df["Scenario"]==idx[1]) &
        (df["Region"]==idx[2]),
    "1750":] = data.sum(skipna=False)["1750":].values

In [ ]:
df.to_csv('rcmip-emissions-annual-means-v6-0-0.csv', index=False)